# Example 04 2.5 duplex

In [1]:
from LoopStructural import GeologicalModel
from LoopStructural.visualisation.model_visualisation import LavaVuModelViewer
import numpy as np
import pandas as pd

In [2]:
boundary_points = np.zeros((2,3))
boundary_points[0,0] = -40
boundary_points[0,1] = -5
boundary_points[0,2] = -10
boundary_points[1,0] = 40
boundary_points[1,1] = 5
boundary_points[1,2] = 10

In [3]:
data = pd.read_pickle('duplex.pkl')
data = data.rename(columns = {"V":"val"})
data.head()

,X,Y,Z,val,type,nx,ny,nz,coord
0,6.100000,0.100000,1.100000,0.0,s0,NaN,NaN,NaN,NaN
1,6.100000,0.100000,2.100000,1.0,s0,NaN,NaN,NaN,NaN
0,-3.041451,0.996086,-0.977360,NaN,s0,0.0,-0.0,1.0,NaN
0,-39.669873,-4.982496,-3.972077,0.0,fault1,NaN,NaN,NaN,0.0
1,-29.669701,-4.986644,-4.047513,0.0,fault1,NaN,NaN,NaN,0.0


In [4]:
data[data['type'] == 's0']

,X,Y,Z,val,type,nx,ny,nz,coord
0,6.100000,0.100000,1.10000,0.0,s0,NaN,NaN,NaN,NaN
1,6.100000,0.100000,2.10000,1.0,s0,NaN,NaN,NaN,NaN
0,-3.041451,0.996086,-0.97736,NaN,s0,0.0,-0.0,1.0,NaN


In [5]:
from scipy.sparse import diags 
def precon(A):
    return diags(1 / A.diagonal(),0)
interpolation_region = lambda pos : pos[:,0] > 0.05

In [6]:
model = GeologicalModel(boundary_points[0,:],boundary_points[1,:])
model.set_model_data(data)
fault1 = model.create_and_add_fault('fault1',
                                    -4,
                                    nelements=5000,
                                    solver='lu',
                                    interpolatortype='FDI',
                                   damp = True,
                                    precon=precon,
                                    buffer=0.15,
#                                                                         region=interpolation_region
                                    steps=10,
                                    cpw=10,
                                    gpw=10
                                   )


In [7]:
# splayregion = lambda pos : ~np.logical_and(fault1['feature'][1].evaluate_value(pos)>-0.25,
#                                            fault1['feature'][1].evaluate_value(pos)<1.5)
# interpolation_region = lambda pos : np.logical_and(fault1['feature'][1].evaluate_value(pos)>-0.3,
#                                            fault1['feature'][1].evaluate_value(pos)<1.55)
splayregion = lambda pos : np.logical_or(pos[:,0] < 0.12,pos[:,0]>.7)
#np.logical_and(fault1['feature'][1].evaluate_value(pos)>-0.3,
                                           #fault1['feature'][1].evaluate_value(pos)<1.55)

In [8]:
# fault2 = model.create_and_add_fault('fault2',
#                                     -6,
#                                     nelements=5000,
#                                     splayregion=splayregion,
#                                     splay=fault1['feature'],
#                                    solver='lu',
#                                    precon=precon,
#                                    interpolatortype='FDI',
#                                    damp=True,
#                                     buffer=0.15,
#                                     cpw=10,
#                                     gpw=10,
# #                                     region=interpolation_region
#                                     steps=10,
#                                    )
s0 = model.create_and_add_foliation('s0',
                                                nelements=2500,
                                                solver='lu',
                                                interpolatortype='FDI',
                                                precon=precon,
#                                                 region=interpolation_region,
                                                damp = True,
                                                buffer=0.15
                                               )
# print(s0.get_interpolator().support.n_nodes)
# s02 = model.create_and_add_conformable_foliation('s0',
#                                                 nelements=2000,
#                                                 solver='lu',
#                                                 interpolatortype='FDI',
#                                                 precon=precon,
# #                                                 region=interpolation_region,
#                                                        damp = True,
#                                                 buffer=0.1
#                                                )
# print(s02.get_interpolator().support.n_nodes)


In [9]:
# splay = RegionFeature(splayregion)
# interp = RegionFeature(interpolation_region)

In [11]:
#%%snakeviz
grid = fault2['feature'][0].get_interpolator().support
viewer = LavaVuModelViewer()
viewer.add_isosurface(fault1['feature'],
#                       slices=[-1,0,1],
                      isovalue=0,
                      voxet=model.voxet()
                     )

# viewer.add_isosurface(fault1['feature'][1],
#                       nslices=5,
#                       voxet=model.voxet())
# viewer.add_isosurface(fault2['feature'][1],
#                       nslices=5,
#                       voxet=model.voxet())
# viewer.add_isosurface(fault2['feature'],
# #                                             slices=[-1,0,1],

#                       isovalue=0,
#                       voxet=model.voxet()
#                      )
# # viewer.add_scalar_field(model.bounding_box,(38,55,30),
# #                       'splay',
# #                      paint_with=splay,
# #                      cmap='prism')
# viewer.add_scalar_field(model.bounding_box,(38,55,30),
#                       'interp',
#                      paint_with=s0['feature'],
#                      cmap='prism')
# # # viewer.add_scalar_field(model.bounding_box,(38,55,30),
# # #                       'box3',
# # #                      paint_with=s0,
# # #                      cmap='prism')
# # # # # viewer.add_scalar_field(model.bounding_box,(38,55,30),
# # # # #                       'box2',
# # # # #                      paint_with=s0,
# # # # #                      cmap='prism')
# # # viewer.add_scalar_field(model.bounding_box,(38,55,30),
# # #                       'box3',
# # #                      paint_with=fault1['feature'],
# # #                      cmap='prism')
# # viewer.add_isosurface(s0,
# #                       isovalue=0,
# # #                       nslices=10,
# #                      voxet=model.voxet()
# #                      )
# # viewer.add_vector_field(fault2['feature'][1],grid.barycentre[::10,:])
# # viewer.add_vector_field(fault1['feature'][1],grid.barycentre[::10,:])

# # viewer.add_isosurface(s02,
# #                       name='test',
# #                       isovalue=0,
# # #                       nslices=10,
# #                      voxet=model.voxet()
# #                      )
# # viewer.add_section()
# # # viewer.add_isosurface(splay,voxet=model.voxet())
# # # viewer.add_data(fault2[0])
viewer.add_data(fault1['feature'][0])
# viewer.add_data(fault2['feature'][0])
# # viewer.add_points(s0.get_interpolator().support.nodes,name='points')
# # viewer.add_points(s02.get_interpolator().support.nodes,name='points2')
# points = fault1['feature'].apply_to_points(grid.nodes)
# # points = fault2['feature'].apply_to_points(points)
# viewer.add_points(points,name='faulted')
# # viewer.add_data(s0)
# # points = fault2['feature'].apply_to_points(points)
# points = fault2['feature'].apply_to_points(points)
# viewer.add_points(points,name='faulted2')

viewer.interactive()

NameError: name 'fault2' is not defined